In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Charger le fichier de Kaggle qui contient les ventes réelles
df_sales = pd.read_excel('data/Pizza_Sales.xlsx')

# Calculer la distribution des pizzas
pizza_distribution = df_sales['pizza_name'].value_counts(normalize=True)

In [2]:
# Créer un fichier d'association pizza-taille-ID
pizza_size_mapping = df_sales[['pizza_name', 'pizza_size', 'pizza_id', 'unit_price']].drop_duplicates()

# Sauvegarder dans un fichier CSV pour réutilisation
pizza_size_mapping.to_csv('data/pizza_size_mapping.csv', index=False, encoding='utf-8')


In [3]:
df_ingredients = pd.read_csv('data/pizza_ingredients.csv')

df_mapping = pd.read_csv('data/pizza_size_mapping.csv')

size_distribution = df_sales.groupby(['pizza_name', 'pizza_size']).size() / len(df_sales)


In [4]:
# Paramètres de simulation
num_clients = 7000 
mean_pizzas_per_client = 7
std_pizzas_per_client = 3  

#  Utiliser une distrib normale pour générer le nombre de pizzas par clients à l'année
np.random.seed(42)  
pizzas_per_client = np.clip(
    np.random.normal(loc=mean_pizzas_per_client, scale=std_pizzas_per_client, size=num_clients),
    2, 20
).astype(int)

In [5]:
# Générer les commandes pour chaque client
data = []
for client_id, num_pizzas in enumerate(pizzas_per_client, start=1):
    favorite_pizzas = pizza_distribution.sample(n=min(2, len(pizza_distribution)), weights=pizza_distribution.values).index.tolist()
    
    for _ in range(num_pizzas):
        order_date = datetime.now() - timedelta(days=random.randint(1, 365))
        
        if num_pizzas <= 7:  # Clients rares
            if random.random() < 0.80:  # 80% chance de pizza favorite
                pizza_name = random.choice(favorite_pizzas)
            else:  # Choisir une pizza aux ingrédients similaires
                favorite_ingredients = df_ingredients[df_ingredients['pizza_name'].isin(favorite_pizzas)]['pizza_ingredients']

                # Créer un ensemble d'ingrédients communs entre toutes les pizzas favorites
                favorite_ingredients_list = favorite_ingredients.str.split(', ').tolist()
                common_ingredients = set(favorite_ingredients_list[0])
                for ingredients in favorite_ingredients_list[1:]:
                    common_ingredients.intersection_update(ingredients)

                # Joindre les ingrédients communs avec '|' pour la recherche
                common_ingredients_str = '|'.join(common_ingredients)

                # Filtrer les pizzas qui contiennent tous les ingrédients communs
                similar_pizzas = df_ingredients[
                    df_ingredients['pizza_ingredients'].str.contains(common_ingredients_str)
                ]
                
                if not similar_pizzas.empty:
                    pizza_name=similar_pizzas.sample(1)['pizza_name'].iloc[0]
                else : 
                    pizza_name=df_ingredients.sample(1)['pizza_name'].iloc[0]

        else:  # Clients fréquents
            if random.random() < 0.25:  # 25% chance de pizza favorite
                pizza_name = random.choice(favorite_pizzas)
            else:
                pizza_name = pizza_distribution.sample(n=1, weights=pizza_distribution.values).index[0]
        
        # Générer la taille de la pizza en respectant la distribution
        possible_sizes = size_distribution.loc[pizza_name]
        pizza_size = possible_sizes.sample(weights=possible_sizes.values).index[0]
        
        # Récupérer le pizza_id et le prix
        pizza_info = df_mapping[(df_mapping['pizza_name'] == pizza_name) & (df_mapping['pizza_size'] == pizza_size)]
        pizza_id = pizza_info['pizza_id'].iloc[0]
        
        # Récupérer les ingrédients
        ingredients = df_ingredients[df_ingredients['pizza_name'] == pizza_name]['pizza_ingredients'].iloc[0]
        
        # Ajouter l'entrée
        data.append({
            'client_id': client_id,
            'pizza_id': pizza_id,
            'pizza_name': pizza_name,
            'pizza_size': pizza_size,
            'order_date': order_date,
            'ingredients': ingredients
        })

# Convertir en DataFrame
df_consumers = pd.DataFrame(data)

# Sauvegarder
df_consumers.to_csv('data/consumer_data.csv', index=False, encoding='utf-8')

In [6]:
# Calculer le nombre total de pizzas par client
pizza_count_per_client = df_consumers.groupby('client_id').size()

# Compter combien de clients ont consommé chaque quantité
distribution = pizza_count_per_client.value_counts().sort_index()

# Afficher les résultats
distribution_df = distribution.reset_index()
distribution_df.columns = ['Number of Pizzas', 'Number of Clients']

print(distribution_df)

    Number of Pizzas  Number of Clients
0                  2                658
1                  3                458
2                  4                659
3                  5                824
4                  6                920
5                  7                917
6                  8                819
7                  9                643
8                 10                470
9                 11                291
10                12                177
11                13                 92
12                14                 44
13                15                 16
14                16                  9
15                17                  1
16                18                  2


In [7]:
# Calculer combien de clients uniques ont consommé chaque pizza
pizza_client_count = df_consumers.groupby('pizza_name')['client_id'].nunique()

# Convertir en DataFrame pour une meilleure lisibilité
pizza_client_distribution = pizza_client_count.reset_index()
pizza_client_distribution.columns = ['Pizza Name', 'Number of Clients']

# Trier par le nombre de clients
pizza_client_distribution = pizza_client_distribution.sort_values(by='Number of Clients', ascending=False)

print(pizza_client_distribution)

                                    Pizza Name  Number of Clients
0                   The Barbecue Chicken Pizza               1449
7                     The Classic Deluxe Pizza               1403
20                         The Pepperoni Pizza               1352
30                      The Thai Chicken Pizza               1334
4                 The California Chicken Pizza               1334
12                          The Hawaiian Pizza               1321
26                     The Spicy Italian Pizza               1179
23                          The Sicilian Pizza               1128
25                 The Southwest Chicken Pizza               1121
9                        The Four Cheese Pizza               1107
14                   The Italian Supreme Pizza               1105
1                           The Big Meat Pizza               1080
17                          The Mexicana Pizza               1031
18                        The Napolitana Pizza               1010
31        